<a href="https://colab.research.google.com/github/harjeet88/google_collab/blob/main/spark_exmples/second_highest_salary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!chmod -R +x /content/drive/MyDrive/installs/

In [3]:
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']
# Verify installation
!java -version
!which java

openjdk version "11.0.2" 2019-01-15
OpenJDK Runtime Environment 18.9 (build 11.0.2+9)
OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
/content/drive/MyDrive/installs/jdk-11.0.2//bin/java


In [4]:
import pyspark
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ["SPARK_HOME"] = "drive/MyDrive/installs/spark-3.5.1-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/installs/py_package') # Replace with your actual path

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
MAX_MEMORY="8g"
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.0.0rc1",
    "org.xerial:sqlite-jdbc:3.36.0.3",
    "graphframes:graphframes:0.8.2-spark3.2-s_2.12",
    "com.acervera.osm4scala:osm4scala-spark3-shaded_2.12:1.0.8",
]

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .enableHiveSupport()
    .getOrCreate()
    )

In [8]:
spark

In [12]:
!ls -ltr /content/drive/MyDrive/data/strata

total 2
-rw------- 1 root root 1465 Nov 18 12:55 employee.csv
-rw------- 1 root root  382 Nov 18 13:26 kpmg_2nd_highest_salary.csv


In [20]:
df = spark.read.csv("/content/drive/MyDrive/data/strata/employee.csv",inferSchema=True,header=True)

In [21]:
df.show()

+---+---------+-------------------+------------+---------+---------------+-------------+-----------+
| id|worker_id|    login_timestamp|  ip_address|  country|         region|         city|device_type|
+---+---------+-------------------+------------+---------+---------------+-------------+-----------+
|  1|      101|2023-11-18 10:00:00| 192.168.1.1|    India|          Delhi|        Delhi|    Desktop|
|  2|      102|2023-11-18 11:00:00| 192.168.1.2|    India|         Mumbai|       Mumbai|     Mobile|
|  3|      103|2023-11-18 12:00:00| 192.168.1.3|      USA|     California|San Francisco|     Laptop|
|  4|      104|2023-11-18 13:00:00| 192.168.1.4|       UK|        England|       London|     Tablet|
|  5|      105|2023-11-18 14:00:00| 192.168.1.5|Australia|New South Wales|       Sydney|    Desktop|
|  6|      106|2023-11-18 15:00:00| 192.168.1.6|   Canada|        Ontario|      Toronto|     Mobile|
|  7|      107|2023-11-18 16:00:00| 192.168.1.7|  Germany|        Bavaria|       Munich|   

In [22]:
df.groupBy("country").count().show()

+---------+-----+
|  country|count|
+---------+-----+
|  Germany|    2|
|   France|    2|
|    India|    4|
|    Italy|    2|
|    Spain|    2|
|      USA|    2|
|       UK|    2|
|   Canada|    2|
|Australia|    2|
+---------+-----+



In [23]:
df.groupBy("region").count().show()

+---------------+-----+
|         region|count|
+---------------+-----+
|         Madrid|    2|
|       Lombardy|    2|
|         Mumbai|    2|
|        Bavaria|    2|
|New South Wales|    2|
|  Île-de-France|    2|
|        Ontario|    2|
|          Delhi|    2|
|        England|    2|
|     California|    2|
+---------------+-----+



In [24]:
df1 = spark.read.csv("/content/drive/MyDrive/data/strata/kpmg_2nd_highest_salary.csv",inferSchema=True,header=True)

In [25]:
df1.show()

+----------+-----------+------+----------+----+
|mployee_id| department|salary| hire_date|rank|
+----------+-----------+------+----------+----+
|        10|Engineering| 60000|2019-02-09|   1|
|        18|         HR|120000|2016-05-26|   1|
|        17|         HR| 90000|2017-12-19|   2|
|        23|         HR| 75000|2017-01-18|   3|
|         4|  Marketing|120000|2017-05-07|   1|
|        19|  Marketing|120000|2019-09-03|   1|
|        11|  Marketing|110000|2015-01-23|   2|
|        25|  Marketing|110000|2019-05-05|   2|
|        21|  Marketing|110000|2017-09-03|   2|
|        12|  Marketing| 90000|2016-12-01|   3|
|        31|  Marketing| 90000|2015-07-10|   3|
+----------+-----------+------+----------+----+



In [29]:
df1.printSchema()

root
 |-- mployee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- rank: integer (nullable = true)



In [30]:
df1.groupBy("department").count().show()

+-----------+-----+
| department|count|
+-----------+-----+
|Engineering|    1|
|         HR|    3|
|  Marketing|    7|
+-----------+-----+



In [36]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,dense_rank

In [48]:
windowSpec= Window.partitionBy("department").orderBy("salary","hire_date")

In [49]:
df1.withColumn("cus_rank",dense_rank().over(windowSpec)).show()

+----------+-----------+------+----------+----+--------+
|mployee_id| department|salary| hire_date|rank|cus_rank|
+----------+-----------+------+----------+----+--------+
|        10|Engineering| 60000|2019-02-09|   1|       1|
|        23|         HR| 75000|2017-01-18|   3|       1|
|        17|         HR| 90000|2017-12-19|   2|       2|
|        18|         HR|120000|2016-05-26|   1|       3|
|        31|  Marketing| 90000|2015-07-10|   3|       1|
|        12|  Marketing| 90000|2016-12-01|   3|       2|
|        11|  Marketing|110000|2015-01-23|   2|       3|
|        21|  Marketing|110000|2017-09-03|   2|       4|
|        25|  Marketing|110000|2019-05-05|   2|       5|
|         4|  Marketing|120000|2017-05-07|   1|       6|
|        19|  Marketing|120000|2019-09-03|   1|       7|
+----------+-----------+------+----------+----+--------+



In [50]:
df2 =df1.withColumn("cus_rank",dense_rank().over(windowSpec))
#.filter("cus_rank=2").show()
df2.show()

+----------+-----------+------+----------+----+--------+
|mployee_id| department|salary| hire_date|rank|cus_rank|
+----------+-----------+------+----------+----+--------+
|        10|Engineering| 60000|2019-02-09|   1|       1|
|        23|         HR| 75000|2017-01-18|   3|       1|
|        17|         HR| 90000|2017-12-19|   2|       2|
|        18|         HR|120000|2016-05-26|   1|       3|
|        31|  Marketing| 90000|2015-07-10|   3|       1|
|        12|  Marketing| 90000|2016-12-01|   3|       2|
|        11|  Marketing|110000|2015-01-23|   2|       3|
|        21|  Marketing|110000|2017-09-03|   2|       4|
|        25|  Marketing|110000|2019-05-05|   2|       5|
|         4|  Marketing|120000|2017-05-07|   1|       6|
|        19|  Marketing|120000|2019-09-03|   1|       7|
+----------+-----------+------+----------+----+--------+



In [51]:
df2.filter(df2.cus_rank==2).show()

+----------+----------+------+----------+----+--------+
|mployee_id|department|salary| hire_date|rank|cus_rank|
+----------+----------+------+----------+----+--------+
|        17|        HR| 90000|2017-12-19|   2|       2|
|        12| Marketing| 90000|2016-12-01|   3|       2|
+----------+----------+------+----------+----+--------+



In [61]:
df.createOrReplaceTempView("df1_temp")

In [65]:
!pip install --upgrade delta-spark